In [1]:
%cd '..'

/mnt/scratch/v_kulikov/histonedb_Vladimir/CURATED_SET


In [2]:
import inspect
import os
import re
import sys


import pandas as pd
from Bio import Entrez, SeqIO
from curated_set_services import CuratedSet
Entrez.email = "v.kulikov@intbio@org"

In [3]:
# Load data from histones.csv
curated_set = CuratedSet()
cs = curated_set

cs.data.shape, cs.data.columns

((567, 16),
 Index(['accession', 'type', 'variant_group', 'variant', 'doublet', 'gi',
        'ncbi_gene_id', 'hgnc_gene_name', 'taxonomy_id', 'organism', 'phylum',
        'class', 'taxonomy_group', 'info', 'references', 'sequence'],
       dtype='object'))

In [4]:
list(cs.has_duplicates())

[]

In [5]:
cs.data[cs.data["accession"] == "AAB48832.1"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence


In [6]:
cs.data.head()

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
NP_505463.1,NP_505463.1,H2A,cH2A,cH2A_(Animals),,17562014,,,6239,Caenorhabditis elegans,Nematoda,Chromadorea,,,26989147 22650316(?),MSGRGKGGKAKTGGKAKSRSSRAGLQFPVGRLHRILRKGNYAQRVG...
EEC09557.1,EEC09557.1,H2A,cH2A,cH2A_(Animals),,215500063,,,6945,Ixodes scapularis,Arthropoda,Arachnida,,,26989147 22650316(?),MSGRGKGGKVKGKSKTRSSRAGLQFPVGRIHRLLRKGNYAERVGAG...
NP_724343.1,NP_724343.1,H2A,cH2A,cH2A_(Animals),,24585673,,,7227,Drosophila melanogaster,Arthropoda,Insecta,,,26989147 22650316(?),MSGRGKGGKVKGKAKSRSNRAGLQFPVGRIHRLLRKGNYAERVGAG...
XP_001119899.1,XP_001119899.1,H2A,cH2A,cH2A_(Animals),,110764935,,,7460,Apis mellifera,Arthropoda,Insecta,,,26989147 22650316(?),MSGRGKGGKAKAKAKSRSNRAGLQFPVGRIHRLLRKGNYAERVGAG...
EDO48405.1,EDO48405.1,H2A,cH2A,cH2A_(Animals),,156227602,,,45351,Nematostella vectensis,Cnidaria,Anthozoa,,,26989147 22650316(?),MSGRGKGKAKGTKSKTRSSRAGLQFPVGRIHRHLRKGNYAERVGAG...


In [7]:
cs.data.shape

(567, 16)

In [8]:
cs_h2b_accessions = [
    "AAB48832.1",
]

In [9]:
df = pd.DataFrame(
    {
        "accession": cs_h2b_accessions,
        "type": ["H2B"] * len(cs_h2b_accessions),
        "variant_group": ["CS_H2B_(Echinoidea)"] * len(cs_h2b_accessions),
        "variant": ["CS_H2B_(Echinoidea)"] * len(cs_h2b_accessions),
        "references": ["9032246"] * len(cs_h2b_accessions),
    }
)
df.index = df.accession
df.shape, df.columns

((1, 5),
 Index(['accession', 'type', 'variant_group', 'variant', 'references'], dtype='object'))

In [10]:
df.head()

,accession,type,variant_group,variant,references
accession,,,,,
AAB48832.1,AAB48832.1,H2B,CS_H2B_(Echinoidea),CS_H2B_(Echinoidea),9032246


In [11]:
cs.data = pd.concat([cs.data, df]).fillna("")
cs.data.shape, list(cs.has_duplicates())

((568, 16), [])

In [12]:
cs.data.tail()

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
NP_005314.2,NP_005314.2,H1,H1.6,H1.6_(Homo_sapiens)__???,,,3010.0,H1-6,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MSETVPAASASAGVAAMEKLPTKKRGRKPAGLISASRKVPNLSVSK...
NP_861453.1,NP_861453.1,H1,H1.7,H1.7_(Homo_sapiens)__???,,,341567.0,H1-7,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MEQALTGEAQSRWPRRGGSGAMAEAPGPSGESRGHSATQLPAEKTV...
NP_005316.1,NP_005316.1,H1,H1.1,H1.1_(Homo_sapiens)__???,,,3024.0,H1-1,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MSETVPPAPAASAAPEKPLAGKKAKKPAKAAAASKKKPAGPSVSEL...
NP_005310.1,NP_005310.1,H1,H1.2,H1.2_(Homo_sapiens)__???,,,3006.0,H1-2,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MSETAPAAPAAAPPAEKAPVKKKAAKKAGGTPRKASGPPVSELITK...
AAB48832.1,AAB48832.1,H2B,CS_H2B_(Echinoidea),CS_H2B_(Echinoidea),,,,,,,,,,,9032246,


In [13]:
cs.data[cs.data["variant"] == "CS_H2B_(Echinoidea)"].shape

(1, 16)

In [14]:
cs.update_accession_version()
cs.data = cs.data.set_index(cs.data.accession.values)

/home/l_singh/.conda/envs/histdb_env/lib/python3.8/site-packages/Bio/GenBank/__init__.py:1143: BiopythonParserWarning: Dropping bond qualifier in feature location
  warnings.warn(
/mnt/scratch/v_kulikov/histonedb_Vladimir/CURATED_SET/curated_set_services.py:267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updating_data['accession'] = new_accessions


In [15]:
cs.data[cs.data["variant"] == "CS_H2B_(Echinoidea)"].shape

(1, 16)

In [16]:
curated_set.update_taxids(blank_data=True)

Fetched taxid from NCBI 7660
 changes to 7660
 changes to Psammechinus miliaris
 changes to Echinodermata
 changes to Echinoidea


In [17]:
cs.data[cs.data["variant"] == "CS_H2B_(Echinoidea)"].shape

(1, 16)

In [18]:
cs.data[cs.data["variant"] == "CS_H2B_(Echinoidea)"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
AAB48832.1,AAB48832.1,H2B,CS_H2B_(Echinoidea),CS_H2B_(Echinoidea),,,,,7660,Psammechinus miliaris,Echinodermata,Echinoidea,,,9032246,


## Updating sequences for CS_H2B

In [19]:
cs.data[cs.data['sequence'] == '']

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
AAB48832.1,AAB48832.1,H2B,CS_H2B_(Echinoidea),CS_H2B_(Echinoidea),,,,,7660,Psammechinus miliaris,Echinodermata,Echinoidea,,,9032246,


In [20]:
cs.update_sequence(blank_data=True)

Fetching 1 seqs
Sequence for AAB48832.1 changes from  to MPAKGAATKGEKKQAVKSKAMASSRTGDKKRRRRRLESYNIYIYKVLKQVHPDTGISSKAMSIMNSFVNDIFERIAAEASRLAQYNKKSTISSREVQTAVRLLLPGELAKHAVSEGTKAVTKYTTSR
Sequences updated: 1


In [21]:
cs.data[cs.data["variant"] == "CS_H2B_(Echinoidea)"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
AAB48832.1,AAB48832.1,H2B,CS_H2B_(Echinoidea),CS_H2B_(Echinoidea),,,,,7660,Psammechinus miliaris,Echinodermata,Echinoidea,,,9032246,MPAKGAATKGEKKQAVKSKAMASSRTGDKKRRRRRLESYNIYIYKV...


In [22]:
cs.save()

Added sequence with accession AAB48832.1
cp histones.csv backups/histones.csv-Apr0224180339
Previous data backuped to backups/histones.csv-Apr0224180339
Results saved to histones.csv
